# Deploying models to Azure Managed Endpoints using Azure Machine Learning

Following this [tutorial](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?tabs=python). Note that this is the most up-to-date method using the Python SDK v2 `azure-ai-ml`, and all tutorials using `azureml-core` (e.g. [dp100](https://github.com/MicrosoftLearning/mslearn-dp100/blob/main/09%20-%20Create%20a%20Real-time%20Inferencing%20Service.ipynb) or [this demo](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb)) are outdated, see [this blog](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-migrate-from-v1) for more info. 

Steps:
1. Setup Azure Machine Learning workspace and Azure Managed Online Endpoint.
2. Deploy model
3. Test model
4. Delete endpoint

Take model from:

In [ ]:
model_path = "../models/model.pkl"

In [ ]:
workspace = "azure-ml-deployment-methods"
resource_group = "azure-cognitive-services-accelerator"

## 1. Setup
### 1.1 Setup Azure Machine Learning 
First [create a new Azure Machine Learning workspace in the Azure portal](https://portal.azure.com/#view/Microsoft_Azure_MLTeamAccounts/CreateMachineLearningServicesBladeV2).

In [ ]:
subscription_id = !az account show --query id

Connect to Azure ML workspace

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

client = MLClient(DefaultAzureCredential(), subscription_id[0][1:-1], resource_group, workspace)

### 1.2 Setup Azure Managed Online Endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

endpoint = ManagedOnlineEndpoint(
    name="sample-model-endpoint",
    description="A sample ML model deployment to Managed Endpoints",
    auth_mode="key",
    tags={'area': 'diabetes', 'type': 'regression'},
)

In [ ]:
client.online_endpoints.begin_create_or_update(endpoint).result()

## 2. Deploy model to endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment, Model, Environment, CodeConfiguration

Configure deployment: select environment and model. Here we use a base Ubuntu image and add conda dependencies.

In [ ]:
!mkdir files

In [ ]:
%%writefile files/inference.py
import json, joblib, os
import numpy as np

def init():
    global model
    model = joblib.load(os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl'))

def run(raw_data):
    predictions = model.predict(np.array(json.loads(raw_data)['data'])).tolist()
    return json.dumps(predictions)

In [ ]:
%%writefile files/conda.yml
name: model-env
channels:
  - defaults
dependencies:
  - python=3.7
  - pip
  - pip:
    - scikit-learn
    - joblib
    - azureml-defaults
    - inference-schema[numpy-support]

In [ ]:
model = Model(path="../models/model.pkl")

env = env = Environment(
    conda_file="files/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

code_config = CodeConfiguration(code="files", scoring_script="inference.py")

Create deployment and connect to endpoint

In [ ]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name="sample-model-endpoint",
    model=model,
    environment=env,
    code_configuration=code_config,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

Deploy. This creates a deployment which you can view in `Azure Machine Learning Studio`/`Endpoints`/`<name>`

In [ ]:
client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
endpoint.traffic = {"blue": 100}
client.begin_create_or_update(endpoint)

## 3. Test endpoint

Grab API key from `Azure Machine Learning Studio`/`Endpoints`/`<name>`/`Consume`

In [ ]:
uri = client.online_endpoints.get(name="sample-model-endpoint").scoring_uri
key = ""

In [56]:
import requests
import json
import numpy as np

input_payload = json.dumps({
    'data': np.load("../data/diabetes.npz")["X_test"][:2].tolist(),
})

requests.post(uri, input_payload, headers={
    'Content-Type':'application/json', 
    'Authorization':('Bearer '+key)
}).json()

'[230.86996560914233, 241.27351292981544]'

## 4. Delete

In [ ]:
client.online_endpoints.begin_delete(name="sample-model-endpoint")

In [ ]:
!rm -r files

.....................................